# Coroutines. Into the Async

![](./images/into_the_unknown.jpeg)

# Содержание<a id="contents"></a>

* [Coroutines](#coroutines)
  * [Debt from the Lab 1: GeneratorExit](#debt1)
  * [Generator's next vs. Coroutine's send](#next-vs-send)
  * [Debt from the Lab 2: Receive & Send](#debt2)
* [CPU & I/O Bound Programs](#cpu-and-io-bound)
* [Deeper in Concurrency: Coroutines vs. Threading vs. Parallelism](#deeper-in-concurrency)
* [Asyncio, Event Loop, Futures (Tasks)](#asyncio)
  * [Butterbrot](#butterbrot)
* [References](#references)
  * [Advanced](#references-advanced)
  * [Related](#references-related)
  * [Probably Also Worth Attention](#references-probably-worth)
  * [Miscellaneous](#references-misc)
  * [Probably Miscellaneous (but Maybe Important)](#references-probably-misc)

In [ ]:
import asyncio
import datetime
import time

from typing import Awaitable

# Need this stuff to run asyncio in Jupyter Notebook:
# https://github.com/ipython/ipython/issues/11338
# https://stackoverflow.com/a/56434301/8094251

import nest_asyncio

nest_asyncio.apply()

## Coroutines<a id="coroutines"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

![](./images/beazley_intro_slide.png)

Приставка "co" в слове корутина — означает "совместно", "вместе".
Ключевую роль играет возможность передачи значения другим корутинам (`send`) и ожидание результата работы от других корутин (`yield from`).

Из [словаря](https://en.wiktionary.org/wiki/coroutine):
> *Coroutine* is a piece of code that performs a task, and that can be passed new input and return output more than once.

И в качестве более частных разновидностей корутины можно выделить *сабрутины* (обычные программы):
> Subroutine is a coroutine that accepts input once and returns output once.

и *генераторы*
> Generator is a coroutine that accepts input once, but yields output multiple times.

А вообще, на корутины и обычные функции можно смотреть просто как на абстракции стека вызовов и порядка выполнения инструкций: когда есть некоторый порядок выполнения блоков кода и в каждом блоке известна текущая выполняемая инструкция.

<figure style="text-align: center;">
    <img src='./images/call_stack.png' alt='missing' />
    <figcaption>Про Call stack на примере JS (<a>https://ru.hexlet.io/courses/js-asynchronous-programming/lessons/call-stack/theory_unit</a>).</figcaption>
</figure>

Как в случае сабрутин, так и в случае корутин, при их выполнении выделяется память под локальные переменные, последовательно выполняются строки, а на стадии `return` возвращаемое значение кладётся в стек вызовов, и освобождается память, выделенная под локальные переменные, под указатель на текущую инструкцию в теле функции.
Отличие же корутин от обычных функций в том, что при выбрысывании значения вовне с помощью `yield` оно кладётся в стек, но *состояние корутины сохраняется*.
При получении значения из `yield` состояние стека восстанавливается, и продолжается выполнение корутины.
Таким образом, корутина может передавать контроль другой функции с сохранением собственного состояния.
Корутина использует часть функционала от генератора, расширяя его.

Сходство корутин с простыми функциями в том, что порядок выполнения кода в обоих случаях последовательный (выполняется одна функция, после неё выполняется другая, но не случайная, а конкретная).
Таким образом, корутины, запущенные из одного источника, могут одновременно находиться в состоянии выполнения, но в каждый момент времени реально выполняется только одна из них ("concurrent but not parallel").

Корутина не может передавать управление произвольной корутине, в то же время находящейся в состоянии исполнения.
Корутина может передать управление либо вновь созданной корутине (`yield from`), либо коду, вызвавшему корутину (который в стеке вызовов находится сразу перед корутиной).
И родитель корутины может уже передать контроль другой дочерней корутине.

### Debt from the Lab 1: GeneratorExit<a id="debt1"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Значения посылаются в корутины с помощью метода `send`.
Закрыть корутину от получения значений можно с помощью метода `close`.
При этом внутри корутины выбрасывается ошибка `GeneratorExit`.
Стоит отметить, что данная ошибка наследуется не от `Exception`, а от `BaseException`, так как "[по сути не является ошибкой](https://docs.python.org/3/library/exceptions.html#GeneratorExit)".

In [ ]:
def receive_value():
    while True:
        try:
            value = yield
        except GeneratorExit as exception:
            print('Coroutine closed!')
            
            raise exception

In [ ]:
coroutine = receive_value()

next(coroutine)

coroutine.send(123)
coroutine.close()

Coroutine closed!


### Generator's `next` vs. Coroutine's `send`<a id="next-vs-send"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Хотя функция `next` получает значение из генератора, а метод `send` посылает значение в корутину, `yield`, который под капотом, *всегда и посылает, и получает значение*.
Здесь важен порядок отправки-получения: в выражении с `yield` значение сначала выбрасывается из корутины вовне, и только потом значение выражения с `yield` становится равным тому значению, которое передано в корутину с помощью `send`.
```python
received_value = yield emitted_value
```

Метод `send` также двоякий: он не только отправляет одно значение, но и принимает другое из корутины.
```python
# coroutine = ...

received_value = coroutine.send(emitted_value)
```

Отсюда становится понятно, почему при инициализации корутины необходимо первый раз сделать `next()` или `send(None)`: после этого происходит первый выброс значения из корутины и выполнение корутины останавливается перед получением значения в `yield`.
Далее, при следующем вызове `send` с уже нормальным значением выполнение корутины продолжается на этапе *получения значения из yield*, продолжается до следующего `yield`, из него выбрасывается значение, и код останавливается, опять, на моменте получения значения в новый `yield`.

С генераторами всё похоже, только там не надо делать "инициализацию", потому что первый выброс значения уже, как правило, осмысленный в случае генератора.

### Debt from the Lab 2: Receive & Send<a id="debt2"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Можно написать корутину, которая и принимает и отдаёт значения.
Правда, это **не рекомендуется**, потому что не понятно и легко ошибиться.

In [ ]:
def receive_and_send():
    value = 'initial value'
    
    while True:
        receved_value = yield value
        
        print(f'Sent: {value}. Received: {receved_value}')
        
        value = receved_value

In [ ]:
coroutine = receive_and_send()

next(coroutine)

'initial value'

In [ ]:
coroutine.send(-17)

Sent: initial value. Received: -17


-17

In [ ]:
coroutine.send("What should I do I'm just a little baby")

Sent: -17. Received: What should I do I'm just a little baby


"What should I do I'm just a little baby"

## CPU & I/O Bound Programs<a id="cpu-and-io-bound"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Зачем нужны корутины?
Корутины — как способ асинхронно выполнять ряд действий.

Вообще, при написании кода, даже до написания, надо задумываться о том
1. Будет ли код ожидать чего-то, например данных из базы данных или от сервера. Если ответ "да", то производительность кода, скорее всего, ограничена операциями ввода-вывода (I/O bound code).
<figure style="text-align: center;">
    <img src='./images/website-loading3.jpg' alt='missing' />
    <figcaption><a>https://www.calvyn.com/6-tips-on-how-to-optimize-page-speed/</a>.</figcaption>
</figure>

2. Будет ли код выполнять какие-то сложные долгие вычисления? Если ответ "да", то, скорее всего, производительность кода ограничена количеством ядер процессора (CPU bound code).

```python
for i in range(10000000000000):
    _ = 1 + 1
```

Ввиду такого свойства текущей реализации CPython языка Питон, как [GIL](https://en.wikipedia.org/wiki/Global_interpreter_lock), CPU bound программы никак не получится оптимизировать, кроме как использовать библиотеку `multiprocessing` для параллельного выполнения кода на нескольких ядрах (которая просто запускает по отдельному интерпретатору Питона на каждом ядре). 
Что же до I/O bound программ, то тут на помощь как раз могут прийти корутины (и не только).

## Deeper in Concurrency: Coroutines vs. Threading vs. Parallelism<a id="deeper-in-concurrency"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

В Питоне есть несколько способов *одновременно* выполнять несколько вещей:
* нити (потоки)
* процессы
* задачи (можно думать о задачах в рамках конкретной библиотеки `asyncio`; основано на корутинах)

В целом подходы могут показаться похожими, но если копать глубже, то у каждого есть свои уникальные черты и область применимости.
Так, потоки отличаются от задач порядком выполнения: остановка и продолжение потоков регулируется операционной системой, в то время как в `asyncio` задачи кооперируют друг с другом: каждая может давать другим понять, что она готова продолжать работу.
Ещё преимущество `asyncio` перед потоками в том, что при выполнении корутин дополнительные расходы по памяти меньше.
При самом создании корутины никакого исполнения вообще не происходит: объект корутины может быть передан окрестратору (Event Loop), который уже может запустить корутину на выполнение тотчас же или позже.

Далее мы подробнее обсудим библиотеку `asyncio` (так как тема лабы — корутины).

## Asyncio, Event Loop, Futures (Tasks)<a id="asyncio"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Библиотека `asyncio` появилась в Питоне ~3.4.
Постоянно дополнялась, обновлялась (некоторые удобные абстракции работают начиная с Питона 3.7).

Плюсы `asyncio`по сравнению с чистыми корутинами:
* С помощью специального синтаксиса корутины выделяются в языке более явно: так, это уже не просто "расширенная версия геренатора".
* Есть готовая реализация Event Loop, который, как планировщик задач или дирижёр, вызывает те корутины-задачи, которые уже готовы продолжать работу. Event Loop ждёт, пока не произойдёт какое-либо событие (что-то скачалось, данные подгрузились, какая-то задача завершилась, ...), и реагирует по ситуации.

<figure style="text-align: center;">
    <img src='./images/event_loop.png' alt='missing' />
    <figcaption>Tasks в Event Loop (<a>https://docs.python.org/3.4/library/asyncio-task.html</a>).</figcaption>
</figure>

* Есть возможность использовать асинхронность в контекстных менеджерах и при итерациях в цикле `for`

Корутина, начиная с Питона ~3.4, модет определаться так
```python
async def read_data(database):
    pass
```

Ключевое здесь — `async def`.
Модификатор `async` делает функцию асинхронной и позволяет использовать внутри такой (и только такой) функции конструкцию с `await`:
```python
async def read_data(database):
    data = await database.fetch('SELECT name FROM mushrooms WHERE cap_color = "RED";')
    
    # Doint something with the data...
```

При этом `await` служит аналогом `yield from`.
И после `await` также может идти не что угодно, а специальный объект (Awaitable объект, который реализуется специальный magic метод).
При использовании `await` вне `async def` функций будет ошибка `SyntaxError`.
Именно `await` передаёт контроль из корутины вызывающему корутину коду и позволяет переключаться на выполнение других задач, пока текущая задача под `await` не будет выполнена.
При определении корутины с помощью `async def` использовать `yield` внутри уже нельзя: как способы возвращения значения вызывающему коду могут использоваться только `return` и `await`.

Корутины в рамках `asyncio` — как задачи (`Task`), которые регулирует Event Loop.
При этом часто для задач можно встретить обозначение `Future`: что-то, что ещё не готово, но посчитается когда-то в будущем (класс `Task` наследуется от класса `Future`).
Метод для создания задачи `create_task` появился позже, чем метод для создания фьючи `create_future` (в Питоне 3.7).

Хоть код читается как "последовательность действий", как и в случае с просто корутинами, но, благодаря орекстратору Event Loop, который решает, какая задача будет выполняться следующей, код на самом деле может выполняться в неочевидном порядке.
Всё зависит от времени завершения конкрентных задач.

Асинхронность важна во многих приложениях.
Так, игра на мобильном телефоне должна давать вам как пользователю возможность выйти из приложения, даже если в этот момент вы управляете бегающей сосиской.
Или при скачивании чего-либо на компьютере: экран остаётся кликабельным, и вы даже можете прервать загрузку.
Упомянутые процессы не блокируют работу: приложение всегда способно обрабатывать другие действия пользователя.
Или при работе сервера (например почтового, Gmail) сервер никогда целиком не переключается на работу с конкретным пользователем: только при попытке выполнить какое-то действие со стороны пользователя (например при отправке сообщения) сервер откликнется на запрос, но при бездействии со стороны пользователя (а пользователь, как правило, большую часть времени бездействует) сервер переключится на работу с кем-то ещё.

Преимущество же синхронного кода по сравнению с асинхронным — в простоте разработки, в понятности и читаемости кода.
Писать асинхронный код тяжелее, так как это требует более глубокого понимания происходящих внутри процессов и того, как они будут взаимодействовать между собой.

Рассмотрим пару небольших примеров с `asyncio`

In [ ]:
async def hello_world():
    print('Hello')
    
    await asyncio.sleep(2)
    
    print('world!')

Не такая уж и корутина:

In [ ]:
coroutine = hello_world()

try:
    next(coroutine)
except TypeError as error:
    print(error)

'coroutine' object is not an iterator


Запустим несколько одинаковых задач асинхронно.
Видно, что никакая задача не блокирует поток:

In [ ]:
task = asyncio.ensure_future(hello_world())
task = asyncio.ensure_future(hello_world())
task = asyncio.ensure_future(hello_world())

Hello
Hello
Hello
world!
world!
world!


И ещё один вариант асинхронного вывода строки `"Hello world!"` (источник: http://larsyencken.github.io/gitbook-mcpy/async/asyncio.html): выводится `"Hello"`, а в это же время "выполняется" функция, возвращающая строку `"world!"`.
Как только второе слово готово, оно тоже выводится.

In [ ]:
async def print_hello_world():
    world = asyncio.ensure_future(get_world())

    await asyncio.sleep(1)

    print('Hello ', end='', flush=True)
    print(await world)


async def get_world():
    await asyncio.sleep(2)
    return 'world!'


loop = asyncio.get_event_loop()
loop.run_until_complete(print_hello_world())

Hello world!


Ладно, это было совсем по-игрушечному.
Надо посмотреть что-то поинтереснее...

### Butterbrot<a id="butterbrot"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

<figure style="text-align: center;">
    <img src='./images/poirot.png' alt='missing' />
    <figcaption><a>https://www.pinterest.com.au/pin/628392954236615095/</a>.</figcaption>
</figure>

Далее идёт переведённый с C# пример синхронного и асинхронного приготовления бутерброда.
Оригинал доступен по ссылке: https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/async.

Ниже определяется пайплайн приготовления завтрака `create_breakfast`.
Классы для составляющих завтрак пустые, так как они важны только в плане идеи.
Функция `sleep` представляет собой симуляцию затрат времени на приготовление.

Сначала рассмотрим последовательное приготовление ("засунули хлеб в тостер, стоим и ждём").

In [ ]:
def create_breakfast():
    cup = pour_coffee()
    print("Coffee is ready")

    eggs = fry_eggs(2)
    print("Eggs are ready")

    bacon = fry_bacon(3)
    print("Bacon is ready")

    toast = toast_bread(2)
    apply_butter(toast)
    apply_jam(toast)
    print("Toast is ready")

    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")


class Coffee:
    pass

class Egg:
    pass

class Bacon:
    pass

class Toast:
    pass

class Juice:
    pass

class OrangeJuice(Juice):
    pass


def sleep():
    num_seconds = 3

    time.sleep(num_seconds)


def pour_coffee() -> Coffee:
    print("Pouring coffee")

    return Coffee()


def fry_eggs(num_eggs: int) -> Egg:
    print("Warming the egg pan...")
    sleep()
    sleep()
    
    print(f"Cracking {num_eggs} eggs")
    
    print("Cooking the eggs ...")
    sleep()
    
    print("Putting eggs on plate")

    return Egg()


def fry_bacon(num_slices: int) -> Bacon:
    print(f"Putting {num_slices} slices of bacon in the pan")
    
    print("Cooking first side of bacon...")
    sleep()
    
    for _ in range(num_slices):
        print("Flipping a slice of bacon")

    print("Cooking the second side of bacon...")
    sleep()
    
    print("Putting bacon on plate")

    return Bacon()


def toast_bread(num_slices: int) -> Toast:
    for _ in range(num_slices):
        print("Putting a slice of bread in the toaster")

    print("Start toasting...")
    sleep()

    print("Remove toast from toaster")

    return Toast()


def apply_butter(toast: Toast) -> None:
    print("Putting butter on the toast")


def apply_jam(toast: Toast) -> None:
    print("Putting jam on the toast")

    
def pour_orange_juice() -> OrangeJuice:
    print("Pouring orange juice")

    return OrangeJuice()

Функция-измеритель времени, затрачиваемого на приготовление завтрака (время считается по результатам одного запуска).

In [ ]:
def measure_time(func):
    start_time = datetime.datetime.now()
    
    func()
    
    finish_time = datetime.datetime.now()
    elapsed_time = finish_time - start_time
    
    print(f'\nElapsed time: {elapsed_time}')

Сколько времени занимает приготовление завтрака:

In [ ]:
measure_time(
    create_breakfast
)

Pouring coffee
Coffee is ready
Warming the egg pan...
Cracking 2 eggs
Cooking the eggs ...
Putting eggs on plate
Eggs are ready
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Putting bacon on plate
Bacon is ready
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Toast is ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:18.001853


Теперь — асинхронное (типа) приготовление:

In [ ]:
async def create_breakfast_async():
    cup = pour_coffee()
    print("Coffee is ready")

    eggs = await fry_eggs_async(2)
    print("Eggs are ready")

    bacon = await fry_bacon_async(3)
    print("Bacon is ready")

    toast = await toast_bread_async(2)
    apply_butter(toast)
    apply_jam(toast)
    print("Toast is ready")

    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")


async def sleep_async():
    num_seconds = 3

    await asyncio.sleep(num_seconds)


async def fry_eggs_async(num_eggs: int) -> Awaitable[Egg]:
    print("Warming the egg pan...")
    await sleep_async()
    await sleep_async()
    
    print(f"Cracking {num_eggs} eggs")
    
    print("Cooking the eggs ...")
    await sleep_async()
    
    print("Putting eggs on plate")

    return Egg()


async def fry_bacon_async(num_slices: int) -> Awaitable[Bacon]:
    print(f"Putting {num_slices} slices of bacon in the pan")
    
    print("Cooking first side of bacon...")
    await sleep_async()
    
    for _ in range(num_slices):
        print("Flipping a slice of bacon")

    print("Cooking the second side of bacon...")
    await sleep_async()
    
    print("Putting bacon on plate")

    return Bacon()


async def toast_bread_async(num_slices: int) -> Awaitable[Toast]:
    for _ in range(num_slices):
        print("Putting a slice of bread in the toaster")

    print("Start toasting...")
    await sleep_async()

    print("Remove toast from toaster")

    return Toast()

Смотрим время...

In [ ]:
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        create_breakfast_async()
    )

measure_time(main)

del main

Pouring coffee
Coffee is ready
Warming the egg pan...
Cracking 2 eggs
Cooking the eggs ...
Putting eggs on plate
Eggs are ready
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Putting bacon on plate
Bacon is ready
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Toast is ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:18.004444


Время не изменилось!
Дело в том, что просто вставки `async` и `await` в исходный код ничего фактически не меняют.
Надо "включить асинхронную логику".
Ниже мы это и сделаем!
Надо создать задачи, но ждать их завершения сразу не надо: пусть играют на фоне, а дождаться их мы должны только тогда, когда нам нужен "осязаемый" результат.

In [ ]:
async def create_breakfast_v2_async():
    """Concurrent breakfast
    """
    cup = pour_coffee()
    print("Coffee is ready")

    eggs_task = asyncio.ensure_future(
        fry_eggs_async(2)
    )
    bacon_task = asyncio.ensure_future(
        fry_bacon_async(3)
    )
    toast_task = asyncio.ensure_future(
        toast_bread_async(2)
    )
    
    toast = await toast_task
    apply_butter(toast)
    apply_jam(toast)
    print("Toast is ready")
    
    eggs = await eggs_task
    print("Eggs are ready")
    
    bacon = await bacon_task
    print("Bacon is ready")

    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")

И время...

In [ ]:
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        create_breakfast_v2_async()
    )

measure_time(main)

del main

Pouring coffee
Coffee is ready
Warming the egg pan...
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Toast is ready
Cracking 2 eggs
Cooking the eggs ...
Putting bacon on plate
Putting eggs on plate
Eggs are ready
Bacon is ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:09.004394


...уменьшилось! Ура!

Но это не предел улучшениям: пара синхронных одномоментных действий (намазывание масла и джема на хлеб) завязаны на одно асинхронное (приготовление хлеба в тостере).
Итого все эти действия вместе представляют собой асинхронный блок.
Давайте объединим всё, связанное с хлебом, в один асинхронный блок!

In [ ]:
async def create_breakfast_v3_async():
    """Concurrent and recomposed breakfast
    """
    cup = pour_coffee()
    print("Coffee is ready")

    eggs_task = asyncio.ensure_future(
        fry_eggs_async(2)
    )
    bacon_task = asyncio.ensure_future(
        fry_bacon_async(3)
    )
    toast_task = asyncio.ensure_future(
        toast_bread_with_butter_and_jam_async(2)
    )
    
    eggs = await eggs_task
    print("Eggs are ready")
    
    bacon = await bacon_task
    print("Bacon is ready")

    toast = await toast_task
    print("Toast is ready")
    
    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")


async def toast_bread_with_butter_and_jam_async(num_slices: int) -> Toast:
    toast = await toast_bread_async(num_slices)
    
    apply_butter(toast)
    apply_jam(toast)

    return toast

Есть ли улучшения?..

In [ ]:
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        create_breakfast_v3_async()
    )

measure_time(main)

del main

Pouring coffee
Coffee is ready
Warming the egg pan...
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Cracking 2 eggs
Cooking the eggs ...
Putting bacon on plate
Putting eggs on plate
Eggs are ready
Bacon is ready
Toast is ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:09.003816


Увы, в данном случае они слишком малы, чтобы мы могли увидеть разницу.
Но это не значит, что программу не стоит продолжать улучшать :)
Ещё один важный пункт, достойный улучшения — ожидание выполнения задач.
Раньше мы ждали их в произвольном порядке (просто, как вздумалось).
Есть лучший способ — использовать функционал `asyncio` для получения задач просто по мере готовности (то есть порядок выполнения задач определяется просто с помощью Event Loop-а).
Ниже — финальный вариант приготовления завтрака:

In [ ]:
async def create_breakfast_v5_async():
    """Concurrent and recomposed breakfast, with improved await
    """
    cup = pour_coffee()
    print("Coffee is ready")

    loop = asyncio.get_event_loop()

    eggs_task = asyncio.ensure_future(
        fry_eggs_async(2)
    )
    bacon_task = asyncio.ensure_future(
        fry_bacon_async(3)
    )
    toast_task = asyncio.ensure_future(
        toast_bread_with_butter_and_jam_async(2)
    )
    
    tasks = [eggs_task, bacon_task, toast_task]
    tiny_timeout = 0.01
        
    while len(tasks) > 0:
        finished_tasks, unfinished_tasks = loop.run_until_complete(
            asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
        )

        for task in finished_tasks:
            assert task.done()

            if task is eggs_task:
                print("Eggs are ready")
            elif task is bacon_task:
                print("Bacon is ready")
            elif task is toast_task:
                print("Toast is ready")

            tasks.remove(task)
    
    eggs = eggs_task.result()
    bacon = bacon_task.result()
    toast = toast_task.result()

    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")


async def toast_bread_with_butter_and_jam_async(num_slices: int) -> Toast:
    toast = await toast_bread_async(num_slices)
    
    apply_butter(toast)
    apply_jam(toast)

    return toast

И время в лучшем случае...

In [ ]:
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        create_breakfast_v5_async()
    )

measure_time(main)

del main

Pouring coffee
Coffee is ready
Warming the egg pan...
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Toast is ready
Cracking 2 eggs
Cooking the eggs ...
Putting bacon on plate
Bacon is ready
Putting eggs on plate
Eggs are ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:09.001273


...изменилось не сильно, по сравнению с предыдущими асинхронными выполнениями.
Увы, улучшения — а они должны быть :) — слишком малы, чтобы их можно было увидеть за один прогон программы.
В оригинальном примере время ожидания измерялось десятками минут (хотя в примерах кода стоят ожидания в $3$ секунды, как в ноутбуке).
Столько ждать не хочется) хотя интересно бы было проверить... но, кажется, разница между async реализациями не зависит от времени одного `sleep`?..

## References<a id="references"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

* [video] https://youtu.be/kdzL3r-yJZY

Классная получасовая лекция про асинхронность в целом.
И там тоже есть пример с приготовлением еды :)

* https://realpython.com/python-concurrency/

Классно, понятно, с примерами (про корутины + нити + параллельность).

* https://ru.stackoverflow.com/a/902762/252621

Прикольно, понятно, на русском, в двух словах про то, что такое `Future` и `Task`.

* [c#] https://channel9.msdn.com/Series/Three-Essential-Tips-for-Async/Three-Essential-Tips-For-Async-Introduction

Несколько видео про C#, но в самом первом видео понятно и общо рассказывают про суть Async (видео на пять минут).

* https://tech.chefclub.tv/an-asyncio-io-bound-producer-playing-with-a-cpu-bound-consumer

В целом прикольно, несложно, упоминают CPU и I/O bound code.
Есть неплохой пример по `asyncio` про consumer + producer (скачивание видео файлов и их обработка).

* http://www.blog.pythonlibrary.org/2016/07/26/python-3-an-intro-to-asyncio/

Тоже в целом про `asyncio`, тоже с примером (только скачивание файлов).

* https://stackoverflow.com/a/34101652/8094251

Объясняют разницу между `yield` из генератора и `send` в корутину (что в каком порядке там происходит).

* https://stackoverflow.com/a/51116910/8094251

Тоже говорят про `yield` в двух ипостасях, про принятие и отправку значений (но как-то размыто и много текста).

* https://stackoverflow.com/a/39792393/8094251

Общий комментарий про то, что такое корутины.

* https://stackoverflow.com/questions/59260268/is-there-ever-a-reason-to-return-await-in-python-asyncio

Заметка про `return await`.

* https://habr.com/ru/post/314606/

Пара заметок про `asyncio` из опыта разработчика в продакшене.
Есть слова о правильном закрытии Event Loop-а (о том, что его надо закрывать).


### Advanced<a id="references-advanced"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

* https://stackoverflow.com/a/51177895/8094251

Мозговыносящий комментарий-статья на StackOverflow.
В первой части объясняются идейные отличия нитей, процессов и задач.
Есть несложный пример кастомного Event Loop (!)

* https://softwareengineering.stackexchange.com/a/352263/348557

Небольшой экскурс в историю корутин (в разных языках программирования).

* https://www.python.org/dev/peps/pep-0492/

PEP 492 — Coroutines with async and await syntax.
Про то, как это всё происходило в Питоне.


### Related<a id="references-related"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

* https://realpython.com/python-gil/

Про [GIL](https://ru.wikipedia.org/wiki/Global_Interpreter_Lock).


### Probably Also Worth Attention<a id="references-probably-worth"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Статьи/видео ниже сам не смотрел, но некоторые из них точно заслуживают внимания.
Ссылки в этом разделе — это фактически TODO автора ноутбука на посмотреть/почитать :)

* [David M. Beazley 1](http://www.dabeaz.com/generators/)

Generator Tricks for Systems Programmers, 2008.


* [David M. Beazley 2](http://dabeaz.com/coroutines/)

A Curious Course on Coroutines and Concurrency, 2009.

Классная презентация.
Живая, с юмором и т.д.
Есть пример про смешивание генерации и принятие значений в корутине.
До конца пока не дошёл...

* [David M. Beazley 3](http://www.dabeaz.com/finalgenerator/)

Generators: The Final Frontier, 2014.

* https://realpython.com/async-io-python/

Подробнее про `asyncio`, в начале тоже есть слова про отличия подходов к параллельности в Питоне.

* [c#][video] https://channel9.msdn.com/Series/Three-Essential-Tips-for-Async/Three-Essential-Tips-For-Async-Introduction

Ещё где-то $6$ небольших видео про C#.

* [video] https://www.youtube.com/watch?v=9zinZmE3Ogk

Raymond Hettinger, Keynote on Concurrency, PyBay 2017.

* [video] https://www.youtube.com/watch?v=E-1Y4kSsAFc

Fear and Awaiting in Async: A Savage Journey to the Heart of the Coroutine Dream.

* http://dabeaz.blogspot.com/2010/01/python-gil-visualized.html

Про GIL.

* [video] https://www.youtube.com/watch?v=Obt-vMVdM8s

Про GIL.

* https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/

Большая статья про `async` + `await` + с примером Event Loop своими руками в конце.

* https://effectivepython.com/2015/03/10/consider-coroutines-to-run-many-functions-concurrently

Пример с Conway’s Game of Life, построенной на корутинах.

* https://towardsdatascience.com/cpython-internals-how-do-generators-work-ba1c4405b4bc

Про геренаторы, есть слова про Call Stack и фреймы.

* http://www.andy-pearce.com/blog/posts/2016/Jun/the-state-of-python-coroutines-yield-from/

Пара подстатей про `asyncio`.

* https://medium.com/python-pandemonium/asyncio-coroutine-patterns-beyond-await-a6121486656f

Бесконечный пост про `asyncio`.
Видимо, какой-то пример.

* https://habr.com/ru/post/420129/

Перевод на русский поста выше.

* https://sahandsaba.com/understanding-asyncio-node-js-python-3-4.html

* https://natenka.github.io/python/fluent-python-coroutine/

На русском.

* http://larsyencken.github.io/gitbook-mcpy/async/asyncio.html


### Miscellaneous<a id="references-misc"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

* [c#] https://docs.microsoft.com/en-us/dotnet/csharp/async

Про Async в C#.

* [js] https://medium.com/devschacht/javascript-eventloop-explained-f2dcf84e36ee

Прикольный пример из JS из серии "2 + 2, но не всё так просто".

* [js] https://habr.com/ru/post/461401/

Классный пример про Event Loop в JS.


### Probably Miscellaneous (but Maybe Important)<a id="references-probably-misc"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Ресурсы ниже тоже не смотрел, но, может, стоит.

* [js][video] https://www.youtube.com/watch?v=8aGhZQkoFbQ

Видео про Event Loop в JS.


* https://www.ibm.com/cloud/learn/virtualization-a-complete-guide

Про виртуализацию.

* https://www.ibm.com/cloud/learn/virtual-machines

Про виртуальные машины.

* https://www.ibm.com/cloud/learn/hypervisors

Про гипервизоры (к виртуализации).

* http://www.kegel.com/c10k.html